In [20]:
import pandas as pd
import ast
import os
from pathlib import Path

def load_hyperparameters(model_name, dataset):
    """
    Load hyperparameters for a specific model and dataset from CSV files in the directory structure:
    <dataset>/Scores and Hyperparams/<model_name>/*.csv
    
    Parameters:
    model_name (str): Name of the model (e.g., 'efficientvit_l1')
    dataset (str): Name of the dataset (e.g., 'msld', 'msid', 'caap')
    
    Returns:
    dict: Dictionary with required hyperparameters
    """
    # Define the expected hyperparameters and their types
    expected_params = {
        'EPOCH': int,
        'batch_size_train': int,
        'batch_size_val': int,
        'batch_size_test': int,
        'learning_rate': float,
        'weight_decay': float,
        'patience': int,
        'min_delta': float,
        'scheduler_gamma': float,
        'dropout': float,
        'neurons1': int,
        'neurons2': int,
        'neurons3': int,
        'neurons4': int,
        'n_layers': int,
        'dilation': ast.literal_eval,
        'loss': str,
        'lr': float,
        
    }
    
    # Construct the path to the model's directory
    base_path = Path(dataset) / 'Scores and Hyperparams' / model_name
    
    # Check if directory exists
    if not base_path.exists():
        raise FileNotFoundError(f"Directory not found: {base_path}")
    
    # Find all CSV files in the directory
    csv_files = list(base_path.glob('*.csv'))
    
    if not csv_files:
        raise FileNotFoundError(f"No CSV files found in {base_path}")
    
    # Use the first CSV file found
    csv_path = csv_files[0]
    print(f"Loading hyperparameters from: {csv_path}")
    
    # Read the CSV file
    df = pd.read_csv(csv_path)
    
    # Initialize the hyperparameters dictionary
    hyperparams = {}
    
    # Get the first row of data
    row = df.iloc[0]
    
    # Extract parameters and convert to appropriate types
    for param, param_type in expected_params.items():
        if param in row:
            try:
                if param == 'dilation':
                    hyperparams[param] = ast.literal_eval(str(row[param]))
                else:
                    hyperparams[param] = param_type(row[param])
            except (ValueError, KeyError, SyntaxError) as e:
                print(f"Warning: Could not convert {param}. Error: {e}")
    
    return hyperparams

# Example usage:
load_hyperparameters('coatnet', 'MCSI')

Loading hyperparameters from: MCSI/Scores and Hyperparams/coatnet/results_20241210_224219_coatnet_1_rw_224.sw_in1k.csv


{'weight_decay': 0.01,
 'neurons1': 1280,
 'neurons2': 32,
 'n_layers': 1,
 'loss': 'FCE',
 'lr': 6.134047274e-05}

In [ ]:
"""
EfficientViT-CAGA: Main Configuration File
This file contains the primary configuration and execution logic for the EfficientViT-CAGA model,
including hyperparameter settings, model configurations, and training execution across different datasets.

Authors: Ayush Deshmukh
Date: 27th Dec
"""

import logging
from typing import Dict, Tuple, Union

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Hyperparameter configurations for MSLD and MSID datasets
HYPERPARAMS_MSLD_MSID = {
    # Training parameters
    'EPOCH': 30,
    'batch_size_train': 16,
    'batch_size_val': 16,
    'batch_size_test': 8,
    'learning_rate': 0.0000614447274,
    'weight_decay': 0.01,
    'patience': 7,
    'min_delta': 0,
    'scheduler_gamma': 0.95,
    
    # Model architecture
    'dropout': 0.111975,
    'neurons1': 1024,
    'neurons2': 2048,
    'neurons3': 512,
    'neurons4': 512,
    'n_layers': 2,
    'dilation': (1, 2),  # Dilations for CAGA module
}

# Hyperparameter configurations for MCSI dataset
HYPERPARAMS_MCSI = {
    # Training parameters
    'lr': 0.0000648047274,
    'weight_decay': 0.01,
    
    # Model architecture
    'neurons1': 1024,
    'neurons2': 2048,
    'n_layers': 2,
    'dilation': (1, 2),  # Dilations for CAGA module
    'loss': 'FCE',  # Options: 'FocalCE', 'CE', 'BCE'
}

# Supported model configurations
SUPPORTED_MODELS = [
    'resnet101.a1_in1k',
    'deit3_medium_patch16_224',
    'coatnet_1_rw_224.sw_in1k',
    'mobilenetv3_large_100.ra_in1k',
    'vit_base_patch16_224',
    'efficientvit_l1.r224_in1k'  # Primary model for EfficientViT-CAGA results
]

# Module configurations for ablation study
MODULE_VARIANTS = {
    'CAGA': 'Complete CAGA module with cascading',
    'CAA': 'Only CAA module without CAGA',
    'CAGA_nocascading_in_CAA': 'CAGA module without cascading in CAA',

}

MODEL_NAMES = [
    'coatnet', 'DEIT3_medium', 'EfficientViT', 'EfficientViT-CAGA', 'mobilenet_large', 'resnet101', 'ViTBase'
    
]

def main(
    dataset: str,
    model_name: str,
    model_config_name: str,
    module: str = 'CAGA',
    pre_trained: bool = True,
    run: int = 1
) -> None:
    """
    Main execution function for training and evaluation across different datasets.
    
    Args:
        dataset: Dataset name ('MCSI', 'MSLD', or 'MSID')
        model_name: Base model architecture name
        model_config_name: Specific model configuration name
        hyperparams: Dictionary containing hyperparameters
        module: Module variant for ablation study ('CAGA', 'CAA', or 'CAGA_nocascading_in_CAA')
        pre_trained: Whether to use pre-trained weights
        run: Run number for experiment tracking
    
    Raises:
        ValueError: If invalid dataset is specified
    """
    if dataset not in ['MCSI', 'MSLD', 'MSID']:
        raise ValueError(f"Invalid dataset: {dataset}. Must be one of: MCSI, MSLD, MSID")
    
   
    logger.info(f"Starting training for {dataset} dataset using {model_name} with {module} module")
    
    if dataset == 'MCSI':
        hyperparams = load_hyperparameters(model_name, dataset)
        %run MCSI_Implementation.ipynb
        training(model_name, model_config_name, hyperparams, module, pre_trained, run=run)
    
    elif dataset == 'MSLD':
        hyperparams = load_hyperparameters(model_name, dataset)
        %run MSLD_Implementation.ipynb
        training(model_name, model_config_name, hyperparams, X_train, y_train, 
                X_val, y_val, X_test, y_test, module, pre_trained, run=run)
    
    elif dataset == 'MSID':
        hyperparams = load_hyperparameters(model_name, dataset)
        %run MSID_implementation.ipynb
        training(model_name, model_config_name, hyperparams, X_train, y_train,
                X_test, y_test, module, pre_trained, run=run)

if __name__ == "__main__":
    # Example usage for EfficientViT-CAGA with ablation study
    main(
        dataset='MSID',
        model_name='EfficientViT-CAGA',
        model_config_name='efficientvit_l1.r224_in1k',
        module= 'CAGA', #use None for all models other than  EfficientViT-CAGA
        pre_trained=True,
        run=1
    )